In [9]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

In [10]:
# 🔧 Metin temizleme fonksiyonu
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [11]:
# 📄 Veri yükle
df = pd.read_csv("books.csv")
df.columns = df.columns.str.strip()
df["Summary"] = df["Summary"].fillna("")
df["Author"] = df["Author"].fillna("Unknown")

In [12]:
# 🧹 Özetleri temizle
df["Cleaned_Summary"] = df["Summary"].apply(preprocess_text)

In [13]:
# 'Vote Count' sütunundaki sayıları temizle
df['Vote Count'] = df['Vote Count'].str.replace(',', '').str.extract(r'(\d+)').astype(float)

# 5000'den fazla oy alanları filtrele
df_filtered = df[df['Vote Count'] > 5000]

In [14]:
# 💬 BERT modeli yükle
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [24]:
# 🔹 Eğer daha önce kaydedilmiş embedding varsa onu yükle
if os.path.exists("embeddings.pt"):
    print("Embedding dosyası bulundu, yükleniyor...")
    embeddings = torch.load("embeddings.pt")
else:
    print("Embedding hesaplanıyor ve kaydediliyor...")
    embeddings = model.encode(df["Cleaned_Summary"].tolist(), convert_to_tensor=True)
    torch.save(embeddings, "embeddings.pt")

Embedding dosyası bulundu, yükleniyor...


C:\Users\sagla\AppData\Local\Temp\ipykernel_8144\2871050199.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("embeddings.pt")


In [25]:
# 📐 Kosinüs benzerliğini hesapla
cosine_sim = cosine_similarity(embeddings.cpu(), embeddings.cpu())

In [26]:
# 🔍 Kitap indeksleri
indices = pd.Series(df.index, index=df["Book Name"].str.lower())

In [27]:
# ✅ Öneri fonksiyonu
def öneri_al(kitap_adi, n=10):
    kitap_adi = kitap_adi.lower()
    if kitap_adi not in indices:
        print("❌ Bu kitap veri setinde bulunamadı.")
        return pd.DataFrame()

    idx = indices[kitap_adi]
    benzerlik_skorlari = list(enumerate(cosine_sim[idx]))
    benzerlik_skorlari = sorted(benzerlik_skorlari, key=lambda x: x[1], reverse=True)
    benzerlik_skorlari = benzerlik_skorlari[1:n+1]

    kitap_indexleri = [i[0] for i in benzerlik_skorlari]
    skorlar = [i[1] for i in benzerlik_skorlari]

    önerilenler = df.iloc[kitap_indexleri][["Book Name", "Author", "Genre", "Rating"]].copy()
    önerilenler["Benzerlik Skoru"] = skorlar
    return önerilenler


In [28]:
# 🧪 Kullanıcıdan kitap al ve önerileri yazdır
kitap = input("Bir kitap adı girin: ")
öneriler = öneri_al(kitap)
if not öneriler.empty:
    print("\n📚 Anlamsal olarak en benzer 10 kitap:")
    print(öneriler.to_string(index=False))


📚 Anlamsal olarak en benzer 10 kitap:
            Book Name             Author       Genre Rating  Benzerlik Skoru
     The Angel's Game  Carlos Ruiz Zafón     Fiction   4.01         0.607879
          Cloud Atlas     David Mitchell     Fiction   4.01         0.598154
         Loving Women        Pete Hamill     Fiction   3.84         0.587004
         East of Eden     John Steinbeck     Fiction   4.42         0.585640
     Jackboot Britain Daniel S. Fletcher     Fiction   3.63         0.577843
The Constant Gardener      John Le Carré     Fiction   3.83         0.575078
 A Dance Through Time       Lynn Kurland Time Travel   4.16         0.568214
Last Exit to Brooklyn   Hubert Selby Jr.     Fiction   3.92         0.567633
Armageddon's Children       Terry Brooks     Fantasy   4.11         0.563481
              Revival       Stephen King      Horror    3.8         0.562574
